Lab 3
------
Welcome to lab 3, in this lab we will be doing the following:
- Retrieving station data from the ECCC API
- Creating thiessen polygons and calculating the area of each polygon
- Gathering data from the ECCC API
- Plotting and visualizing the data


### Library Imports
To run this lab you must have the following packages installed:
- requests (to retrieve data from the API)
- numpy (to manipulate the data)
- pandas (to manipulate the data)
- geopandas (to manipulate the data geospatially)
- folium (for creating interactive maps)
- matplotlib (for plotting and visualizing data)
- shapely (for working with polygons)
- scipy (for creating the Thiessen polygons)

You can install these packages by running the following command in your terminal:
```bash
pip install requests numpy pandas geopandas shapely folium matplotlib shapely scipy
```

In [ ]:
import piplite
await piplite.install("requests")
await piplite.install("numpy")
await piplite.install("pandas")
await piplite.install("geopandas")
await piplite.install("scipy")
await piplite.install("shapely")
await piplite.install("folium")
await piplite.install("matplotlib")

In [ ]:
# Load packages
import io  # Input/Output library, base package already included in Python
import datetime  # Date and time library, base package already included in Python
import requests
import numpy as np
import pandas as pd
import geopandas as gpd
import folium as fm
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from shapely.geometry import Polygon
from scipy.spatial import Voronoi

### Function Documentation and Creation

The next code block contains functions I have created to help you with the lab, this block needs to be executed before you can carry on with the lab. While you do not need to understand the inner workings of these functions, you do need to know their arguments and what they return.

Functions:
- **bbox**: Create a geopandas dataframe with a box around a point
  - Arguments:
    - loc: a geopandas series with a point geometry
    - distance: The distance in meters from the point to the edge of the box (in KM)
  - Returns:
    - A geopandas dataframe with a box around the point
  - NOTE: This function works **REQUIRES** locations in a projected coordinate system
- **retrieve_climate_stations**: Retrieve climate stations from ECCC's API
  - Arguments:
    - location: a geopandas series with a point geometry in a projected coordinate system (e.g. UTM)
    - yearStart: Start year you want to the data to start from
    - yearEnd: End year you want to the data to end at
    - radial_distance: the distance in KM from the point to search for within the radial distance
  - Returns:
    - A geopandas dataframe with the stations in the bounding box that have daily data between both the start and end years, in the WGS84 coordinate system
- **retrieve_weather_data**: Retrieve weather data from ECCC's API
  - Arguments:
    - stations: A dataframe with the stations you want to retrieve the data from (requires the columns 'STN_ID')
    - startDate: Start date you want to the data to start from
    - endDate: End date you want to the data to end at
  - Returns:
    - A pandas dataframe with the weather data
- **retrieve_hydrometric_data**: Retrieve hydrometric stations from ECCC's API
  - Arguments:
    - stationNum: The station number you want to retrieve the data from (e.g. 02HB001)
    - startDate: Start date you want to the data to start from
    - endDate: End date you want to the data to end at
  - Returns:
    - A geopandas dataframe with the flow data from the station

In [ ]:
# Function to create a geopandas dataframe of a bounding box around a location
def bbox(loc, radial_distance):
    # Extract the x and y coordinates of the location
    x, y = loc.values.x.item(), loc.values.y.item()

    # Convert the radial distance to meters
    radial_distance_m = radial_distance * 1000

    # Define the bounding box
    bboxCoords = [x - radial_distance_m, y - radial_distance_m, x + radial_distance_m, y + radial_distance_m]

    # Convert to a polygon
    bboxPoly = Polygon([(bboxCoords[0], bboxCoords[1]), (bboxCoords[0], bboxCoords[3]), (bboxCoords[2], bboxCoords[3]), (bboxCoords[2], bboxCoords[1])])

    # Create a GeoDataFrame
    bbox = gpd.GeoDataFrame(geometry=[bboxPoly], crs=loc.crs)
    bbox = bbox.to_crs(epsg=4326) # Convert to WGS84 (lat/lon)

    return bbox

# Function that queries the weather data from the ECCC API using the bounding box and start and end years
def retrieve_climate_stations(location, yearStart, yearEnd, radial_distance):
    # Get the bounding box
    boxarea = bbox(location, radial_distance + 100) # Add 100 km to the buffer to ensure stations are found
    minx, miny, maxx, maxy = boxarea.bounds.values.tolist()[0]

    # First create the URL for the API using the bounding box
    url = f'https://api.weather.gc.ca/collections/climate-stations/items?bbox={minx},{miny},{maxx},{maxy}&lang=en-CA&f=csv'

    # Get the data from the API and read it into a pandas dataframe
    response = requests.get(url)

    # Using the IO library convert the text into a file-like object and then read it into a pandas dataframe
    stations = pd.read_csv(io.StringIO(response.text))

    # Filter the stations to only include those that have data for the specified years
    stations['DLY_FIRST_DATE'] = pd.to_datetime(stations['DLY_FIRST_DATE'], format='ISO8601')
    stations['DLY_LAST_DATE'] = pd.to_datetime(stations['DLY_LAST_DATE'], format='ISO8601')
    # This will reduce the number of stations to only those that have data for the specified years
    stations = stations[
        (stations['DLY_FIRST_DATE'].dt.year <= yearStart) & (stations['DLY_LAST_DATE'].dt.year >= yearEnd)
        ].reset_index(drop=True)
    
    # Convert the stations to a GeoDataFrame
    stations = gpd.GeoDataFrame(stations, geometry=gpd.points_from_xy(stations.x, stations.y), crs='EPSG:4326')

    # In order to just get the stations that are within the a 25km radius of the basin, we can use a buffer to 
    # reduce the number of stations. We will need to use the projected coordinate system of the basin to do this.
    stations_pcs = stations.to_crs(location.crs)

    # Calculate the distance from the basin centroid to each station
    stations_pcs['distance (km)'] = stations_pcs.distance(location.values[0])/1000

    # Cycle through each station and check if it intersects with the buffer, keep increasing size until at least
    # 10 stations are found
    stationsWithin = pd.DataFrame()
    radius = 25
    while len(stationsWithin) < 10:
        centroid_buffer = location.buffer(radius*1000)
        stationsWithin = stations_pcs[stations_pcs.within(centroid_buffer.iloc[0])].reset_index(drop=True)
        if stationsWithin.empty:
            print(f'No stations found within {radius} km')
        radius += 5

    # Sort the stations by distance, and select the first 10 stations
    stationsWithin = stationsWithin.sort_values('distance (km)')[:10].reset_index(drop=True)

    # Convert the stations back to WGS84
    stationsWithin = stationsWithin.to_crs(epsg=4326)
    
    return stationsWithin

# Function to download the climate data for the specified stations and years
def retrieve_weather_data(stations, startDate, endDate):
    # Create an empty dataframe to store the data
    weatherData = pd.DataFrame()
   
    # Cycle through each year and station to get the data
    startDate = datetime.datetime.strptime(startDate, '%Y-%m-%d')
    endDate = datetime.datetime.strptime(endDate, '%Y-%m-%d')
    currYear = startDate.year
    endYear = endDate.year

    # Using a while loop to cycle through each year
    while currYear <= endYear:
        # Cycle through each station
        for i, stat in stations.iterrows():
            # create the URL for the API using the station ID
            base_url = "https://api.weather.gc.ca/collections/climate-daily/items?f=csv&lang=en-CA"
            url = f"{base_url}&STN_ID={stat['STN_ID']}&LOCAL_YEAR={currYear}&sortby=LOCAL_DATE" # Queries added to the base URL
            
            # Get the data from the API and read it into a pandas dataframe
            response = requests.get(url)
            csvText = str(response.text) # Convert the response to a string to be read by pandas

            # Using the IO library convert the text into a file-like object and then read it into a pandas dataframe
            data = pd.read_csv(io.StringIO(csvText))

            # Concatenate the data to the weatherData dataframe
            weatherData = pd.concat([weatherData, data], ignore_index=True)
        
        # Increment the year
        currYear += 1

    # Convert the date column to a datetime object
    weatherData['DATE'] = pd.to_datetime(weatherData['LOCAL_DATE'], format='ISO8601')

    # Filter the data to only include the dates between the start and end date
    weatherData = weatherData[(weatherData['DATE'] >= startDate) & (weatherData['DATE'] <= endDate)]
        
    return weatherData[['STATION_NAME', 'DATE', 'MEAN_TEMPERATURE', 'TOTAL_PRECIPITATION']] # Return only the columns of interest

# Function to retrieve the hydrometric stations from nearby the basin centroid using the ECCC API
def retrieve_hydrometric_data(stationNum, startDate, endDate):
    # First create the URL for the API using the bounding box
    base_url = "https://api.weather.gc.ca/collections/hydrometric-daily-mean/items?f=csv&lang=en-CA"
    url = f'{base_url}&STATION_NUMBER={stationNum}&datetime={startDate}/{endDate}' # Queries added to the base URL

    # Get the data from the API and read it into a pandas dataframe
    response = requests.get(url)

    # Using the IO library convert the text into a file-like object and then read it into a pandas dataframe
    flowData = pd.read_csv(io.StringIO(response.text))

    # Convert the date column to a datetime object
    flowData['DATE'] = pd.to_datetime(flowData['DATE'], format='ISO8601')

    return flowData[['STATION_NUMBER', 'STATION_NAME', 'DATE', 'DISCHARGE', 'DISCHARGE_SYMBOL_EN']] # Return only the columns of interest

## Retrieving Nearby Stations

In [ ]:
# Read in the shapefile of the basin boundary using geopandas read_file function (gpd.read_file)
basin = ?

# This will be a dataframe with a length of 3
# Plot each row within the basin to look at it
basin.iloc[[?]].plot()

In [ ]:
# Select the actual boundary (correct feature) as the rest are errors generated by the delineation algorithm (review the otputs of the plots from the above cell)
basin = basin.iloc[[?]]

# Save the projected coordinate system of the basin for later use
basin_crs = basin.crs

# Get the centroid of the basin using the .centroid method on the dataframe (dataframe.centroid)
basin_centroid = ?
basin_centroid_coords = basin_centroid.to_crs(epsg=4326)

# Reproject the layer to WGS84 (EPSG Code: 4326) since the weather guages are in WGS84. You can do this by using the to_crs method on the geopandas dataframe
basin = basin.to_crs(epsg=?)

In [ ]:
# Using the predefined functions we will retrieve the climate stations, we should create a bounding box
# large enough to capture plenty of stations around the basin. A suggested value is 125 km, but we also
# need to specify the years we are interested in, in this case 2000 to 2020.
stations = retrieve_climate_stations(basin_centroid, 2000, 2020, 25)

# Display the first 3 stations in the list
stations.head(3)

## Creating Thiessen Polygons for the Basin


In [ ]:
# Thiessen polygons are Voroni diagrams that are used in geophysics to divide a region into polygons
# That is why we will use the scipy library to create the Thiessen polygons (Voroni diagrams) for the stations

# Create a Voronoi diagram using the station locations
vor = Voronoi(stations[['x', 'y']])

# Convert the Voronoi vertices to a list of Polygons
thiessenPoly = [Polygon(vor.vertices[region]) for region in vor.regions if region and -1 not in region]

# Create a GeoDataFrame of the Thiessen polygons
thiessenPoly = gpd.GeoDataFrame(geometry=thiessenPoly, crs='EPSG:4326')

# Clip the Thiessen polygons to a bounding box around the basin
thiessensClip = gpd.clip(thiessenPoly, bbox(basin_centroid, 100))

#Clip the Thiessen polygons to the basin boundary
thiessensBasin = gpd.sjoin(
    thiessensClip,
    basin, 
    predicate='intersects'
).drop(columns=['index_right'])

In [ ]:
# Spatailly join the stations to the Thiessen polygons using a spatial left join to find all stations WITHIN the theissenPolygons
stationsThiessen = gpd.sjoin(stations, thiessenPoly, how='?', predicate='?').rename(columns={'index_right': 'Thiessen_Index'})

# Display the stations with their corresponding Thiessen polygons 
stationsThiessen[['STN_ID', 'STATION_NAME', 'Thiessen_Index']]

In [ ]:
# Test map to see if the stations and Thiessen polygons are correct
m = fm.Map(location=[basin_centroid_coords.y.item(), basin_centroid_coords.x.item()], zoom_start=8)

# Add the basin boundary to the map
fm.GeoJson(
    data = basin,
    name = 'Basin Boundary',
    color = 'red',
    fill = False,
    weight = 2
).add_to(m)

# Add the all Thiessen polygons to the map
fm.GeoJson(
    data = thiessensClip,
    name = 'Thiessen Polygons',
    color = 'black',
    fill = True,
    fill_color = 'black',
    fill_opacity = 0.1,
    weight = 2
).add_to(m)

# Add only the Thiessen polygons that intersect with the basin to the map
fm.GeoJson(
    data = thiessensBasin,
    color = 'blue',
    fill = True,
    fill_color = 'blue',
    fill_opacity = 0.1,
    weight = 2,
).add_to(m)

# Add the stations of interest to the map
for i, stat in stationsThiessen.iterrows():
    # Check if the station has a Thiessen polygon index and add the station to the map
    if pd.notna(stat['Thiessen_Index']):
        fm.Marker(
            location=[stat['y'], stat['x']],
            popup=stat['STATION_NAME'].title(),
            icon=fm.Icon(color='cadetblue')
        ).add_to(m)

# Display the map
m

We can see that there are stations that we can use to gather data from

## Retreiving Climate Data for the Stations from ECCC's API

In [ ]:
# We can now spatially join the stations to the Thiessen polygons that intersect with the basin using an inner join
# This will show only the stations that are within the basin and their corresponding Thiessen polygons
# You need to fill out the left dataframe with the stations and the right dataframe with the Thiessen polygons that are WITHIN the basin
stationsBasin = gpd.sjoin(
    ?,
    ?,
    how='inner', # Returns only values that have a match in both GeoDataFrames
    predicate='?' # This is the spatial relationship used to join the data
).rename(columns={'index_right': 'Thiessen_Index'}).iloc[:,:-5] # Drop the extra columns 

# Display the stations, IDs, names and their corresponding Thiessen polygons index
stationsBasin[['STN_ID', 'STATION_NAME', 'Thiessen_Index']]

In [ ]:
# First we will clip the Thiessen polygons to the basin boundary using the gpd.clip function
# Arguments: The GeoDataFrame to be clipped, The GeoDataFrame to clip with
thiessensBasin = ?

In [ ]:
# We will plot the clipped Thiessen polygons on the map to see the result
m = fm.Map(location=[basin_centroid_coords.y.item(), basin_centroid_coords.x.item()], zoom_start=8)

# Add the basin boundary to the map
fm.GeoJson(
    data = basin,
    name = 'Basin Boundary',
    color = 'red',
    fill = False,
    weight = 2
).add_to(m)

# Add the clipped Thiessen polygons to the map
fm.GeoJson(
    data = thiessensBasin,
    name = 'Thiessen Polygons',
    color = 'black',
    fill = True,
    fill_color = 'black',
    fill_opacity = 0.1,
    weight = 2
).add_to(m)

# Display the map
m

In [ ]:
# To caclulate the area of the Thiessen polygons we need to reproject the polygons to the projected coordinate system of the basin
# then we can calculate the area of the polygons (thiessenBasin) in square meters and join it to the stations on the Thiessen_Index we have created.
stationsBasin = stationsBasin.join(
    # Calculate the area of the Thiessen polygons in square meters
    ?.to_crs(basin_crs).area.rename('Area (m^2)'),
    on='?' # Join the data on the left DataFrame using the Thiessen index column
)

# We can now calculate the weighted area of each the Thiessen polygon by dividing the area of the polygon by the total area of the basin
stationsBasin['Weighted Area'] = stationsBasin['?'] / sum(stationsBasin['?'])

# Display the new columns
stationsBasin[['STN_ID', 'STATION_NAME', 'Thiessen_Index', 'Area (m^2)', 'Weighted Area']]

### Precipitation Data
Here we will be gathering the precipitation data for the stations for the year of 2010. Aftewards we will reformat the data for higher usuability, and visualize it.

In [ ]:
# Using a predefined function we can now retrieve the weather data for the stations within the basin
weatherData = retrieve_weather_data(stationsBasin, '2010-01-01', '2010-12-31')
precip = weatherData[['STATION_NAME', 'DATE', 'TOTAL_PRECIPITATION']]
precip.head()

In [ ]:
# We can now pivot the data to have the date as the index and the station names as the columns, with the total precipitation as the values
# This will reshape the dataframe from long to wide format
precip = pd.pivot_table(precip, index='DATE', columns='STATION_NAME', values='TOTAL_PRECIPITATION')
precip.head()

In [ ]:
# We will now add an additional column to be filled with the basin average precipitation and fill it with empty values
precip['basin_average'] = np.nan
precip.head()

In [ ]:
# We can now calculate the basin average precipitation using the weighted area of each station, while also checking for missing values
for i, row in precip.iterrows():
    # Grab the precipitation values for each station
    pi = row[:-1]

    # Check to see if there is data from all 3 stations
    if pi.count() == 3:
        basin_avg = sum(pi.values * stationsBasin['Weighted Area'].values)
    # Check to see if there is data from 2 stations
    elif pi.count() == 2:
        # If there are 2 stations we will assume each contributes 50% to the basin average
        basin_avg = sum(pi.dropna()) / 2
    # Check to see if there is data from 1 station
    elif pi.count() == 1:
        # If there is only 1 station we will assume it contributes 100% to the basin average
        basin_avg = pi.dropna().values[0]
    # If there is no data from any stations
    else:
        basin_avg = np.nan

    # Fill the basin average column with the calculated value
    precip.loc[i, 'basin_average'] = basin_avg

In [ ]:
# Create a bar plot of the precipitation data
plt.style.use('ggplot') # Set the style of the plot to match one made in R
plt.bar(
    x = precip.index, 
    height = precip['basin_average'],
    color='blue',
    label='Basin Average'
)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b-%y')) # Formates thge x-axis to show the month and year
plt.xlabel('Date')
plt.ylabel('Basin Average Precipitation (mm)')
plt.show() # Display the plot
plt.close() # Close the plot to free up memory and avoid overlapping plots

### Temperature Data
We will now gather the mean temperature data for the stations for the year of 2010. Aftewards we will reformat the data for higher usuability, and visualize it.

In [ ]:
# Using the weather data we can extract the mean temperature for the stations within the basin
temp = weatherData[['STATION_NAME', 'DATE', 'MEAN_TEMPERATURE']]
temp = pd.pivot_table(temp, index='DATE', columns='STATION_NAME', values='MEAN_TEMPERATURE')
temp.head()

In [ ]:
# Create a bar plot of the precipitation data
plt.style.use('ggplot') # Set the style of the plot to match one made in R
plt.bar(
    x = temp.index, 
    height = temp['MEAN_TEMPERATURE'],
    color='red',
    label='Temperature'
)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b-%y')) # Formates thge x-axis to show the month and year
plt.xlabel('Date')
plt.ylabel('Temperature (°C)')
plt.show() # Display the plot
plt.close() # Close the plot to free up memory and avoid overlapping plots

## Retrieving Hydrometric Flow Data for the basin

In [ ]:
# Retrieve the flows for the "Bow River Station" (05BB001) using the retrieve_hydrometric_data function for the year of 2010
flowData = retrieve_hydrometric_data(?)
flowData.head()

In [ ]:
plt.style.use('ggplot') # Set the style of the plot to match one made in R
plt.plot(
    flowData['DATE'],
    flowData['DISCHARGE'],
    color='red',
    label='Discharge'
)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b-%y')) # Formates thge x-axis to show the month and year
plt.xlabel('Date')
plt.ylabel('Discharge (m\u00b3/s)') # Unicode allows for superscript
plt.show() # Display the plot
plt.close() # Close the plot to free up memory and avoid overlapping plots